In [ ]:
import spark.implicits._
import org.apache.spark.sql.functions._
import org.apache.spark.mllib.recommendation.ALS
import org.apache.spark.mllib.recommendation.MatrixFactorizationModel
import org.apache.spark.mllib.recommendation.Rating



  

>     import spark.implicits._
>     import org.apache.spark.sql.functions._
>     import org.apache.spark.mllib.recommendation.ALS
>     import org.apache.spark.mllib.recommendation.MatrixFactorizationModel
>     import org.apache.spark.mllib.recommendation.Rating

In [ ]:
val fileName="dbfs:/FileStore/tables/project4/hetrec2011-lastfm-2k/user_artists.dat"
val df = spark.read.format("csv").option("header", "true").option("delimiter", "\t").option("inferSchema","true").load(fileName).withColumnRenamed("weight","play_count").withColumnRenamed("userID", "listenerID")
df.cache()

  

>     fileName: String = dbfs:/FileStore/tables/project4/hetrec2011-lastfm-2k/user_artists.dat
>     df: org.apache.spark.sql.DataFrame = [listenerID: int, artistID: int ... 1 more field]
>     res7: df.type = [listenerID: int, artistID: int ... 1 more field]

In [ ]:
df.sort(desc("play_count")).show(10)

In [ ]:
val fileName="dbfs:/FileStore/tables/project4/hetrec2011-lastfm-2k/artists.dat"
val artist_names = spark.read.format("csv").option("header", "true").option("delimiter", "\t").option("inferSchema","true").load(fileName).withColumnRenamed("id","artistID").select("artistID","name")
artist_names.cache()

  

>     fileName: String = dbfs:/FileStore/tables/project4/hetrec2011-lastfm-2k/artists.dat
>     artist_names: org.apache.spark.sql.DataFrame = [artistID: int, name: string]
>     res9: artist_names.type = [artistID: int, name: string]

In [ ]:
println("Number of data points: " + df.count())
df.agg(countDistinct("listenerID") as "Number of listeners").show()
df.agg(countDistinct("artistID") as "Number of artists").show()

  

>     Number of data points: 92834
>     +-------------------+
>     |Number of listeners|
>     +-------------------+
>     |               1892|
>     +-------------------+
>
>     +-----------------+
>     |Number of artists|
>     +-----------------+
>     |            17632|
>     +-----------------+

In [ ]:
//Sparsity
println((17632.0*1892.0-92834.0)/(17632.0*1892.0))

  

>     0.9972171848800758

In [ ]:
display(df.select("play_count"))

  

[TABLE]

Truncated to 30 rows

In [ ]:
val artist_data = df.groupBy("artistID").agg(count("artistID") as "unique_listeners",
                                                      sum("play_count") as "total_plays_artist")
artist_data.sort(desc("total_plays_artist")).show(5)
artist_data.sort(desc("unique_listeners")).show(5)

In [ ]:
//Most popular artists with respect to total plays
artist_names.filter($"artistID"===289 || $"artistID"===72 || $"artistID"===89 || $"artistID"===292 || $"artistID"===498).show()

//Most popular artists with respect to unique listeners
artist_names.filter($"artistID"===89 || $"artistID"===289 || $"artistID"===288 || $"artistID"===227 || $"artistID"===300).show()

  

>     +--------+------------------+
>     |artistID|              name|
>     +--------+------------------+
>     |      72|      Depeche Mode|
>     |      89|         Lady Gaga|
>     |     289|    Britney Spears|
>     |     292|Christina Aguilera|
>     |     498|          Paramore|
>     +--------+------------------+
>
>     +--------+--------------+
>     |artistID|          name|
>     +--------+--------------+
>     |      89|     Lady Gaga|
>     |     227|   The Beatles|
>     |     288|       Rihanna|
>     |     289|Britney Spears|
>     |     300|    Katy Perry|
>     +--------+--------------+

In [ ]:
display(artist_data.select("unique_listeners"))

  

[TABLE]

Truncated to 30 rows

In [ ]:
display(artist_data.select("total_plays_artist"))

  

[TABLE]

Truncated to 30 rows

In [ ]:
display(artist_data.select("total_plays_artist", "unique_listeners"))

  

[TABLE]

Truncated to 30 rows

In [ ]:
val user_data = df.groupBy("listenerID").agg(count("listenerID") as "unique_artists",
                                                  sum("play_count") as "total_plays_listener")
user_data.sort(desc("total_plays_listener")).show(5)

  

>     +----------+--------------+--------------------+
>     |listenerID|unique_artists|total_plays_listener|
>     +----------+--------------+--------------------+
>     |       757|            50|              480039|
>     |      2000|            50|              468409|
>     |      1418|            50|              416349|
>     |      1642|            50|              388251|
>     |      1094|            50|              379125|
>     +----------+--------------+--------------------+
>     only showing top 5 rows
>
>     user_data: org.apache.spark.sql.DataFrame = [listenerID: int, unique_artists: bigint ... 1 more field]

In [ ]:
display(user_data.sort(desc("total_plays_listener")))

  

[TABLE]

Truncated to 30 rows

In [ ]:
val df_joined = df.join(artist_data, "artistID").join(user_data, "listenerID").join(artists,"artistID").select("listenerID", "artistID","play_count", "name", "unique_artists","unique_listeners", "total_plays_listener","total_plays_artist")

  

>     df_joined: org.apache.spark.sql.DataFrame = [listenerID: int, artistID: int ... 6 more fields]

In [ ]:
val df_joined_filtered = df_joined.filter($"unique_artists">4 && $"unique_listeners">4)
df_joined_filtered.cache()


  

>     df_joined_filtered: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [listenerID: int, artistID: int ... 6 more fields]
>     res462: df_joined_filtered.type = [listenerID: int, artistID: int ... 6 more fields]

In [ ]:
df_joined_filtered.sort(desc("play_count")).show(10)

In [ ]:
val user_artist=df_joined_filtered.select("listenerID", "artistID", "play_count") //Change DFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
user_artist.show(10)
println("Number of data points: " + user_artist.count())
user_artist.agg(countDistinct("listenerID") as "Number of listeners").show()
user_artist.agg(countDistinct("artistID") as "Number of artists").show()

  

>     +----------+--------+----------+
>     |listenerID|artistID|play_count|
>     +----------+--------+----------+
>     |         2|      51|     13883|
>     |         2|      52|     11690|
>     |         2|      53|     11351|
>     |         2|      54|     10300|
>     |         2|      55|      8983|
>     |         2|      56|      6152|
>     |         2|      57|      5955|
>     |         2|      58|      4616|
>     |         2|      59|      4337|
>     |         2|      61|      3923|
>     +----------+--------+----------+
>     only showing top 10 rows
>
>     Number of data points: 70652
>     +-------------------+
>     |Number of listeners|
>     +-------------------+
>     |               1858|
>     +-------------------+
>
>     +-----------------+
>     |Number of artists|
>     +-----------------+
>     |             2799|
>     +-----------------+
>
>     user_artist: org.apache.spark.sql.DataFrame = [listenerID: int, artistID: int ... 1 more field]

In [ ]:
val user_artistRDD = user_artist.rdd.map(row => Rating(row.getInt(0),row.getInt(1), row.getInt(2)))

  

>     user_artistRDD: org.apache.spark.rdd.RDD[org.apache.spark.mllib.recommendation.Rating] = MapPartitionsRDD[744814] at map at command-2294440354339338:1

In [ ]:
val Array(trainingRDD, validationRDD, testRDD) = user_artistRDD.randomSplit(Array(0.8, 0.2, 0.0), 0L)
trainingRDD.cache()
validationRDD.cache()
testRDD.cache()

  

>     trainingRDD: org.apache.spark.rdd.RDD[org.apache.spark.mllib.recommendation.Rating] = MapPartitionsRDD[744880] at randomSplit at command-3103574048361401:1
>     validationRDD: org.apache.spark.rdd.RDD[org.apache.spark.mllib.recommendation.Rating] = MapPartitionsRDD[744881] at randomSplit at command-3103574048361401:1
>     testRDD: org.apache.spark.rdd.RDD[org.apache.spark.mllib.recommendation.Rating] = MapPartitionsRDD[744882] at randomSplit at command-3103574048361401:1
>     res450: testRDD.type = MapPartitionsRDD[744882] at randomSplit at command-3103574048361401:1

In [ ]:
val rank = 15
val numIterations = 10
val lambda=0.25
val alpha=1.0
val model = ALS.trainImplicit(trainingRDD, rank, numIterations, lambda, alpha)


  

>     rank: Int = 15
>     numIterations: Int = 10
>     lambda: Double = 0.25
>     alpha: Double = 1.0
>     model: org.apache.spark.mllib.recommendation.MatrixFactorizationModel = org.apache.spark.mllib.recommendation.MatrixFactorizationModel@556d74d8

In [ ]:
df_joined_filtered.filter($"listenerID"===1810).sort(desc("play_count")).show(10)

  

>     +----------+--------+----------+--------------+--------------+----------------+--------------------+------------------+
>     |listenerID|artistID|play_count|          name|unique_artists|unique_listeners|total_plays_listener|total_plays_artist|
>     +----------+--------+----------+--------------+--------------+----------------+--------------------+------------------+
>     |      1810|     288|     27486|       Rihanna|             9|             478|               30179|            905417|
>     |      1810|     300|      1850|    Katy Perry|             9|             471|               30179|            532541|
>     |      1810|     475|       358|        Eminem|             9|             202|               30179|            321009|
>     |      1810|     701|       162|       Shakira|             9|             319|               30179|            688529|
>     |      1810|      89|       160|     Lady Gaga|             9|             610|               30179|           1291386|
>     |      1810|     891|       136|         Jewel|             9|              18|               30179|              9412|
>     |      1810|    1760|        12|  Renato Russo|             9|               7|               30179|              3184|
>     |      1810|     466|         8|         Ke$ha|             9|             361|               30179|            384404|
>     |      1810|     289|         7|Britney Spears|             9|             521|               30179|           2393139|
>     +----------+--------+----------+--------------+--------------+----------------+--------------------+------------------+

In [ ]:
trainingRDD.filter{case Rating(listener,artist,plays) => listener==1810}.map{case Rating(listener,artist,plays) =>(artist,plays)}.sortBy(_._2, ascending = false).take(10)

  

>     res457: Array[(Int, Double)] = Array((300,1850.0), (701,162.0), (891,136.0), (1760,12.0), (466,8.0))

In [ ]:
validationRDD.filter{case Rating(listener,artist,plays) => listener==1611}.map{case Rating(listener,artist,plays) =>(artist,plays)}.sortBy(_._2,  ascending = false).take(10)

  

>     res441: Array[(Int, Double)] = Array((2500,651.0))

In [ ]:
val predicted = model.recommendProducts(1810, 10)

//predicted.map{case Rating(user, artist, score) => artist}.zipWithIndex.filter(p => p._1 == 229).take(5)

  

>     predicted: Array[org.apache.spark.mllib.recommendation.Rating] = Array(Rating(1810,333,0.9459016710075482), Rating(1810,89,0.9437156291908422), Rating(1810,679,0.9303599122893169), Rating(1810,300,0.9231183336384853), Rating(1810,331,0.9190269807771281), Rating(1810,298,0.908096096885354), Rating(1810,466,0.9011996354190186), Rating(1810,306,0.8986075315150057), Rating(1810,288,0.892545317423586), Rating(1810,289,0.8892359575673122))

In [ ]:
artists.filter($"artistID"===333 || $"artistID"===89 || $"artistID"===679 || $"artistID"===300).show()

  

>     +--------+-------------+
>     |artistID|         name|
>     +--------+-------------+
>     |      89|    Lady Gaga|
>     |     300|   Katy Perry|
>     |     333|Avril Lavigne|
>     |     679|    Glee Cast|
>     +--------+-------------+

In [ ]:
user_artist.filter($"listenerID"===100 && $"artistID"===424 ).show()

  

>     +----------+--------+----------+
>     |listenerID|artistID|play_count|
>     +----------+--------+----------+
>     +----------+--------+----------+